# 1. Introduction to RNNs
# -----------------------------------------

Recurrent Neural Networks (RNNs) are a class of neural networks designed to recognize patterns in sequences of data, such as time series, text, genomes, handwriting, or the spoken word.

### How RNNs Work
RNNs have a "memory" that captures information about what has been calculated so far. They have loops within the network that allow information to persist.

### Applications of RNNs
- Natural Language Processing (NLP)
- Time Series Forecasting
- Speech Recognition
- Video Processing


# 2. Architecture of RNNs
# -----------------------------------------
"""
### RNN Cells
RNNs are composed of RNN cells. Each cell takes an input vector and the hidden state vector from the previous time step, and produces an output vector and a new hidden state vector.

### Unfolding an RNN
An RNN can be unfolded over time steps to visualize how it processes a sequence. At each time step, the cell receives an input and the hidden state from the previous time step.

### Vanishing Gradient Problem
RNNs can suffer from the vanishing gradient problem, where gradients become very small, slowing down learning and making it hard to capture long-term dependencies.
"""

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler

In [7]:
torch.manual_seed(42)

In [8]:
# Define a simple RNN cell
class SimpleRNNCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SimpleRNNCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.activation = nn.Tanh()

    def forward(self, x, hidden):
        hidden = self.activation(self.i2h(x) + self.h2h(hidden))
        return hidden

In [9]:
# Create a simple RNN sequence
input_size = 1
hidden_size = 5
seq_length = 10


In [10]:
rnn_cell = SimpleRNNCell(input_size, hidden_size)
hidden_state = torch.zeros(hidden_size)

In [5]:
sequence_length = 5
x_seq = [np.random.randn(input_size, 1) for _ in range(sequence_length)]
h_prev = np.zeros((hidden_size, 1))

In [11]:
inputs = torch.randn(seq_length, input_size)

for i, input_t in enumerate(inputs):
    hidden_state = rnn_cell(input_t, hidden_state)
    print(f"Time step {i+1}: {hidden_state}")

Time step 1: tensor([ 0.9073,  0.6078,  0.5593,  0.9744, -0.8246], grad_fn=<TanhBackward0>)
Time step 2: tensor([ 0.9000,  0.0443,  0.7809,  0.9141, -0.4654], grad_fn=<TanhBackward0>)
Time step 3: tensor([ 0.6607, -0.7273,  0.8115,  0.4401, -0.0509], grad_fn=<TanhBackward0>)
Time step 4: tensor([ 0.9163, -0.0936,  0.5820,  0.8935, -0.1833], grad_fn=<TanhBackward0>)
Time step 5: tensor([ 0.8479, -0.4158,  0.7272,  0.7505, -0.2323], grad_fn=<TanhBackward0>)
Time step 6: tensor([ 0.7824, -0.6060,  0.7421,  0.6632, -0.0459], grad_fn=<TanhBackward0>)
Time step 7: tensor([ 0.8868, -0.3241,  0.6497,  0.8257, -0.1300], grad_fn=<TanhBackward0>)
Time step 8: tensor([ 0.6911, -0.7292,  0.7664,  0.4519, -0.0365], grad_fn=<TanhBackward0>)
Time step 9: tensor([ 0.7261, -0.6649,  0.6983,  0.5805, -0.0040], grad_fn=<TanhBackward0>)
Time step 10: tensor([ 0.8792, -0.3193,  0.6395,  0.8296, -0.1580], grad_fn=<TanhBackward0>)


In [12]:
# Generating a sine wave dataset
def generate_sine_wave(seq_length, num_samples):
    x = np.linspace(0, 2 * np.pi, seq_length)
    data = np.sin(x)
    data = np.tile(data, (num_samples, 1))
    return data

In [13]:
seq_length = 50
num_samples = 1000
data = generate_sine_wave(seq_length, num_samples)

In [14]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
data = scaler.fit_transform(data)

In [15]:
data = torch.FloatTensor(data)

In [16]:
class SineWaveDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, index):
        x = self.data[index:index + self.seq_length]
        y = self.data[index + 1:index + self.seq_length + 1]
        return x, y

In [17]:
dataset = SineWaveDataset(data, seq_length)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [18]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, hidden = self.rnn(x)
        out = self.fc(out)
        return out

In [19]:
input_size = 1
hidden_size = 32
output_size = 1

In [21]:
model = RNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [22]:
num_epochs = 50

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs = inputs.view(-1, seq_length, input_size)
        targets = targets.view(-1, seq_length, output_size)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/50], Loss: 0.0532
Epoch [2/50], Loss: 0.0206
Epoch [3/50], Loss: 0.0174
Epoch [4/50], Loss: 0.0153
Epoch [5/50], Loss: 0.0134
Epoch [6/50], Loss: 0.0117
Epoch [7/50], Loss: 0.0102
Epoch [8/50], Loss: 0.0088
Epoch [9/50], Loss: 0.0077
Epoch [10/50], Loss: 0.0067
Epoch [11/50], Loss: 0.0059
Epoch [12/50], Loss: 0.0051
Epoch [13/50], Loss: 0.0045
Epoch [14/50], Loss: 0.0039
Epoch [15/50], Loss: 0.0035
Epoch [16/50], Loss: 0.0030
Epoch [17/50], Loss: 0.0026
Epoch [18/50], Loss: 0.0023
Epoch [19/50], Loss: 0.0020
Epoch [20/50], Loss: 0.0017
Epoch [21/50], Loss: 0.0015
Epoch [22/50], Loss: 0.0013
Epoch [23/50], Loss: 0.0011
Epoch [24/50], Loss: 0.0009
Epoch [25/50], Loss: 0.0008
Epoch [26/50], Loss: 0.0007
Epoch [27/50], Loss: 0.0005
Epoch [28/50], Loss: 0.0004
Epoch [29/50], Loss: 0.0004
Epoch [30/50], Loss: 0.0003
Epoch [31/50], Loss: 0.0002
Epoch [32/50], Loss: 0.0002
Epoch [33/50], Loss: 0.0002
Epoch [34/50], Loss: 0.0001
Epoch [35/50], Loss: 0.0001
Epoch [36/50], Loss: 0.0001
E

In [23]:
# Save the trained RNN model
model_path = 'rnn_model.pth'
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")


Model saved to rnn_model.pth
